# Evaluation of Results of "TUD-MMC at MediaEval 2016: Context of Experience task" by Wang & Liem

## Necessary imports

In [76]:
import pandas as pd
import numpy as np
import os.path
import xml.etree.ElementTree as ET

train_path = "res/coe_dataset_icpr/dev_set/"
test_path = "res/coe_dataset_icpr/test_set/"

audio_folder = "audio_descriptors/"
text_folder = "text_descriptors/"
vis_folder = "vis_descriptors/"
metadata_folder = "XML/"

train_entries_path = "res/CoeTraining.csv"
test_entries_path = "res/CoeTestLabels.csv"
testtrain_entries_path = "res/CoeTrainingTest.csv"

## Datasets

Features are built in the manner described in the paper of Wang & Liem or "Right Inflight? A Dataset for Exploring the Automatic
Prediction of Movies Suitable for a Watching Situation" (https://mmsys2016.itec.aau.at/papers/MMSYS/a45-riegler.pdf), if Wang & Liem do not provide any information.

This leads to following set-up:

Metadata: (language, year published, genre, country, runtime and age rating) - from XML -- 1-Hot Encoding for all categorical values<br>
Text: as is td-idf <br>
Audio: Averaged of all Frames (NaN to 0) - Mel-Frequency Cepstral Coefficients<br>
Visual: as is - Histogram of Oriented Gradients (HOG) gray, Color Moments, local binary patterns (LBP) and Gray Level Run Length Matrix

NOTE: Training data - invalid entry (2_states, also in test set), (Moulin_Rouge!.mp4, should be Moulin_Rouge! --> fixed)

In [59]:
df_base_train = pd.read_csv(train_entries_path)
df_base_train = df_base_train[df_base_train['file_name'] != '2_States'] # remove invalid entry
df_base_train.sort_values(by='file_name', inplace=True)
df_base_train.reset_index(inplace=True, drop=True)
df_targets_train = df_base_train['goodforairplanes'].astype(int)
df_base_train.head(5)

,movie_name,file_name,goodforairplanes
0,A Fish Called Wanda,A_Fish_Called_Wanda,1
1,A Goofy Movie,A_Goofy_Movie,0
2,A Million Ways to Die in the West,A_Million_Ways_to_Die_in_the_West,1
3,A Single Man,A_Single_Man,1
4,American Gangster,American_Gangster,1


In [91]:
df_base_train = pd.read_csv(test_entries_path)
df_base_train = df_base_train[df_base_train['file_name'] != '2_States'] # remove invalid entry

#df_base_train.reset_index(inplace=True, drop=True)
df_base_testtrain = pd.read_csv(testtrain_entries_path)
#df_base_testtrain.sort_values(by='file_name', inplace=True)
df_base_testtrain.reset_index(inplace=True, drop=True)

df_base_train=pd.concat([df_base_train,df_base_testtrain])


df_base_train.sort_values(by='file_name', inplace=True)
df_base_train.reset_index(inplace=True, drop=True)
df_targets_train = df_base_train['goodforairplanes'].astype(int)
df_base_train



,movie_name,file_name,goodforairplanes
0,10.000 Km,10.000_Km,1
1,12 Years a Slave,12_Years_a_Slave,1
2,21 Jump Street,21_Jump_Street,1
3,A Bug's Life,A_Bug's_Life,1
4,A_Fish_Called_Wanda,A_Fish_Called_Wanda,1
5,A Little Chaos,A_Little_Chaos,0
6,A Most Wanted Man,A_Most_Wanted_Man,1
7,A Simple Life,A_Simple_Life,1
8,Aanmodderfakker,Aanmodderfakker,1
9,Alexander,Alexander,1


## Feature extractors
As the dataset was built in a manner that would have been considered dirty already in 2002 a lot of feature extraction is done

In [165]:
def get_audio_features(file_name, use_train=True):
    """
        returns 1x14 dataframe, with averaged Mel-Frequency Cepstral Coefficients + file_name
    """
    base_path = train_path if use_train else test_path
    file_path = os.path.join(base_path, audio_folder, file_name + ".csv")
    if not os.path.isfile(file_path):
        print(file_name, " does not exist!")
        return pd.DataFrame(columns=[str(x) for x in range(13)] + ['file_name'])
    df_audio = pd.read_csv(file_path, header=None).T # transpose (columns are rows)
    df_audio = df_audio.fillna(0) # nan values are treated as 0
    df_audio = pd.DataFrame(df_audio.mean(axis=0)).T # average accross columns
    df_audio['file_name'] = file_name
    return df_audio

def get_all_audio_features(df, use_train=True):
    """
        returns nx14 dataframe, containing audio features for all movies
    """
    dfs = []
    for file_name in df['file_name']:
        dfs.append(get_audio_features(file_name, use_train))
    
    return pd.concat(dfs).reset_index(drop=True)

def get_all_text_features(df, use_train=True):
    """
        returns nx3284 dataframe, containing tf-idf features for all movies
        the dataset creators messed up - contains several terms multiple times
        ordered alphabetically (?) - Live_Nude_Girls and Transformers__Age_of_Extinction where switched (detected perchance)
    """
    base_path = train_path if use_train else test_path
    file_path = os.path.join(base_path, text_folder, "tdf_idf_test.csv")
    df_txt = pd.read_csv(file_path)
    # the creators of the dataset missed how csv-files work - so we transpose and drop empty rows to get the correct format
    cols = df_txt.columns 
    df_txt = df_txt.T.dropna()
    df_txt.columns = cols
    df_txt.reset_index(inplace=True, drop=True)
    df_txt['file_name'] = sorted(df['file_name']) # we assume the info to be order alphabetically, as we do not have more info
    return df_txt.reset_index(drop=True)


def get_vis_features(file_name, use_train=True):
    """
        returns 1x1653 dataframe, with unspecified visual features + file_name
        we assume that every single value in the csv is one feature
        this may be wrong, as there are two rows and no documentation (again)
    """
    base_path = train_path if use_train else test_path
    file_path = os.path.join(base_path, vis_folder, file_name + ".csv")
    if not os.path.isfile(file_path):
        print(file_name, " does not exist!")
        return pd.DataFrame(columns=[str(x) for x in range(1652)] + ['file_name'])
    df_vis = pd.read_csv(file_path, header=None)
    df_vis = pd.DataFrame(pd.concat([df_vis.loc[0,:], df_vis.loc[1,:]])).reset_index(drop=True).T # treat each value as single feature (-> no aggregation)
    df_vis['file_name'] = file_name
    return df_vis

def get_all_vis_features(df, use_train=True):
    """
        returns nx1653 dataframe, containing visual features for all movies
    """
    dfs = []
    for file_name in df['file_name']:
        dfs.append(get_vis_features(file_name, use_train))
    
    return pd.concat(dfs).reset_index(drop=True)

def get_meta_features(file_name, use_train=True):
    """
        returns 1x7 dataframe, with metadata features + file_name
        One Hot Encoding is not applied here, this should happen later
    """
    base_path = train_path if use_train else test_path
    file_path = os.path.join(base_path, metadata_folder, file_name + ".xml")
    if not os.path.isfile(file_path):
        print(file_name, " does not exist!")
        return pd.DataFrame(columns=['country', 'genre', 'language', 'rated', 'runtime', 'year', 'file_name'])
    etree = ET.parse(file_path)
    movie = etree.getroot().find('movie')
    mv = {}
    mv['language'] = [movie.get('language')]
    mv['year'] = [int(movie.get('year'))]
    mv['genre'] = [movie.get('genre')]
    mv['country'] = [movie.get('country')]
    mv['runtime'] = [0 if movie.get('runtime')=='N/A'  else int(movie.get('runtime')[:-4])]
    mv['rated'] = [movie.get('rated')]
    
    df_meta = pd.DataFrame.from_dict(mv)
    df_meta['file_name'] = file_name
    
    return df_meta

def get_meta_features_withrating(file_name, use_train=True):
    """
        returns 1x7 dataframe, with metadata features + file_name
        One Hot Encoding is not applied here, this should happen later
    """
    base_path = train_path if use_train else test_path
    file_path = os.path.join(base_path, metadata_folder, file_name + ".xml")
    if not os.path.isfile(file_path):
        print(file_name, " does not exist!")
        return pd.DataFrame(columns=['country', 'genre', 'language', 'rated', 'runtime', 'year', 'tomatoUserRating','imdbRating','file_name'])
    etree = ET.parse(file_path)
    movie = etree.getroot().find('movie')
    mv = {}
    mv['language'] = [movie.get('language')]
    mv['year'] = [int(movie.get('year'))]
    mv['genre'] = [movie.get('genre')]
    mv['country'] = [movie.get('country')]
    mv['runtime'] = [int(movie.get('runtime')[:-4])]
    mv['rated'] = [movie.get('rated')]
    mv['tomatoUserRating']=  ["NaN" if movie.get('tomatoUserRating')=='N/A'  else movie.get('tomatoUserRating')]
    mv['imdbRating']= [float(movie.get('imdbRating'))]
    df_meta = pd.DataFrame.from_dict(mv)
    df_meta['file_name'] = file_name
    
    return df_meta


def get_all_meta_features(df, use_train=True):
    """
        returns nx7 dataframe, containing metadata features for all movies
    """
    dfs = []
    for file_name in df['file_name']:
        dfs.append(get_meta_features(file_name, use_train))
    
    df_meta = pd.concat(dfs)
    
    df_country = df_meta.country.str.replace(' ','').str.get_dummies(sep=',')
    df_country.columns = ['country_' + x for x in df_country.columns]

    df_genre = df_meta.genre.str.replace(' ','').str.get_dummies(sep=',')
    df_genre.columns = ['genre_' + x for x in df_genre.columns]

    df_language = df_meta.language.str.replace(' ','').str.get_dummies(sep=',')
    df_language.columns = ['language_' + x for x in df_language.columns]

    df_rated = df_meta.rated.str.get_dummies(sep=',')
    df_rated.columns = ['rated_' + x for x in df_rated.columns]
    
    return pd.concat([df_country, df_genre, df_language, df_rated, df_meta[['runtime', 'year', 'file_name']]], axis=1).reset_index(drop=True)

In [93]:
df_txt_train = get_all_text_features(df_base_train,use_train=False)

df_txt_train = df_txt_train[df_txt_train['file_name'] != 'A_Fish_Called_Wanda']
df_txt_train = df_txt_train[df_txt_train['file_name'] != 'Alexander_and_the_Terrible_Horrible_No_Good_Very_Bad_Day']
df_txt_train = df_txt_train[df_txt_train['file_name'] != 'Good_Morning_Vietnam']
df_txt_train = df_txt_train[df_txt_train['file_name'] != 'Love_Rosie']
df_txt_train = df_txt_train[df_txt_train['file_name'] != 'The_Good_the_Bad_and_the_Ugly']
df_txt_train.reset_index(inplace=True, drop=True)

df_base_train = df_base_train[df_base_train['file_name'] != 'A_Fish_Called_Wanda']
df_base_train = df_base_train[df_base_train['file_name'] != 'Alexander_and_the_Terrible_Horrible_No_Good_Very_Bad_Day']
df_base_train = df_base_train[df_base_train['file_name'] != 'Good_Morning_Vietnam']
df_base_train = df_base_train[df_base_train['file_name'] != 'Love_Rosie']
df_base_train = df_base_train[df_base_train['file_name'] != 'The_Good_the_Bad_and_the_Ugly']
df_base_train.reset_index(inplace=True, drop=True)
df_targets_train = df_base_train['goodforairplanes'].astype(int)

In [94]:
df_audio_train = get_all_audio_features(df_base_train,use_train=False)
df_audio_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,file_name
0,-2.526758,-4.041369,1.993339,0.186229,0.439292,-1.073407,-1.204877,-0.040134,0.180450,-2.916251,-0.829277,-0.455054,0.208318,10.000_Km
1,65.069285,-2.643378,0.614012,-0.269641,-0.290764,0.306304,-0.611462,-0.173489,0.961753,-1.172046,-0.519489,-1.271658,-0.179605,12_Years_a_Slave
2,65.721634,-4.928884,0.814478,-0.145326,0.265794,-0.428410,0.121390,-0.313247,0.381455,-0.924582,0.192938,-0.167529,-0.026522,21_Jump_Street
3,60.218722,-7.358705,-0.310818,-1.095195,-0.596027,-0.824320,0.196180,-0.086074,0.384909,-0.763167,-1.051928,-1.290511,-1.151264,A_Bug's_Life
4,64.730372,-2.603121,0.734011,1.156563,-0.798081,-0.453259,-1.149437,0.286334,-1.379145,-0.181773,-1.572061,-1.267311,-0.090285,A_Little_Chaos
5,60.658010,-2.266060,1.657855,1.028910,0.607900,1.251477,-0.105159,-0.173230,-0.261850,-0.321715,0.321455,0.058193,-0.246221,A_Most_Wanted_Man
6,67.085961,1.781262,0.305900,0.746188,-0.257237,-0.200025,0.370250,-0.345818,-2.137183,-1.707853,0.097651,-1.478550,-1.164362,A_Simple_Life
7,65.985640,-2.875637,0.472032,-1.601717,2.256657,0.956259,0.950898,0.492875,0.547208,0.012388,0.721714,-1.461903,1.804050,Aanmodderfakker
8,66.469278,-3.822306,0.180833,-0.654696,-0.329590,-0.805191,-1.568045,-1.727193,-0.556033,-0.790086,0.822247,0.794421,0.353131,Alexander
9,62.779312,-5.516617,-0.241958,-1.305328,-0.184150,0.587740,-0.029263,0.106667,-0.041566,0.198214,-0.302957,0.667275,0.894902,Alice_in_Wonderland


In [101]:
df_vis_train = get_all_vis_features(df_base_train,use_train=False)
df_vis_train

,0,1,2,3,4,5,6,7,8,9,...,1643,1644,1645,1646,1647,1648,1649,1650,1651,file_name
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,269700.0000,6.103500e-07,0.001645,0.000002,0.001645,1638400.0,269700.00,518400.0,269700.00,10.000_Km
1,0.000000,0.193000,0.192990,0.000000,0.000000,0.271100,0.271100,0.000000,0.000000,0.270240,...,15454.0000,1.529400e+01,13.422000,13.418000,13.307000,71633.0,21855.00,48314.0,24434.00,12_Years_a_Slave
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,119790.0000,1.377600e-06,0.002470,0.000004,0.002470,725900.0,119790.00,230400.0,119790.00,21_Jump_Street
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,43.2690,2.964500e+00,3.737800,2.463500,3.952600,4444.8,2405.40,5602.6,2611.10,A_Bug's_Life
4,0.272800,0.260960,0.270090,0.318910,0.296170,0.277970,0.304250,0.357020,0.270390,0.302700,...,60.5890,7.630400e+00,8.487600,7.460300,8.475200,5797.3,460.95,1206.1,458.15,A_Little_Chaos
5,0.000000,0.000000,0.000161,0.000162,0.313130,0.344490,0.341150,0.321080,0.308520,0.339390,...,854.0200,2.176400e+01,16.822000,14.428000,16.660000,259450.0,32323.00,57982.0,32463.00,A_Most_Wanted_Man
6,0.000000,0.000000,0.000000,0.000000,0.339790,0.161840,0.141070,0.224330,0.239770,0.166240,...,7772.2000,1.239100e+01,12.498000,12.388000,12.783000,36773.0,7978.00,13848.0,7937.10,A_Simple_Life
7,0.271970,0.334210,0.316890,0.307720,0.265150,0.276710,0.265630,0.275900,0.240060,0.145130,...,86.8600,6.819900e+00,7.184600,6.196700,7.160700,5318.3,1022.90,3030.9,1012.10,Aanmodderfakker
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,51.9700,7.853800e+00,8.116700,6.557600,7.814300,8186.6,1578.00,2474.1,1736.40,Alexander
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,112920.0000,1.371100e-06,0.002504,0.000005,0.002504,729320.0,112920.00,211600.0,112920.00,Alice_in_Wonderland


In [168]:
df_meta_train = get_all_meta_features(df_base_train,use_train=False)

df_meta_train
#df_combined=pd.merge(df_meta_train, df_base_train, left_index=True, right_index=True)
#df_combined=df_combined.drop(['file_name_x'],axis = 1)
#df_combined=df_combined.drop(['file_name_y'],axis = 1)
#df_combined=df_combined.drop(['movie_name'],axis = 1)

#df_combined=df_combined.loc[:,['tomatoUserRating', 'goodforairplanes']]
#df_combined.to_csv('ratingTomatoes.csv')
#df_combined=pd.merge(df_vis_train, df_combined, left_index=True, right_index=True)
#df_combined=df_combined.drop(['file_name'],axis = 1)
#

,country_Australia,country_Belgium,country_BosniaandHerzegovina,country_Brazil,country_Bulgaria,country_Canada,country_China,country_CzechRepublic,country_Denmark,country_Finland,...,rated_N/A,rated_NOT RATED,rated_PG,rated_PG-13,rated_R,rated_TV-MA,rated_UNRATED,runtime,year,file_name
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,99,2014,10.000_Km
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,134,2013,12_Years_a_Slave
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,109,2012,21_Jump_Street
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,95,1998,A_Bug's_Life
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,117,2014,A_Little_Chaos
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,122,2014,A_Most_Wanted_Man
6,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,118,2011,A_Simple_Life
7,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,100,2014,Aanmodderfakker
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,175,2004,Alexander
9,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,108,2010,Alice_in_Wonderland


## Implementation fun

In [169]:
import random
def sample_gen(n, forbid):
    state = dict()
    track = dict()
    for (i, o) in enumerate(forbid):
        x = track.get(o, o)
        t = state.get(n-i-1, n-i-1)
        state[x] = t
        track[t] = x
        state.pop(n-i-1, None)
        track.pop(o, None)
    del track
    for remaining in range(n-len(forbid), 0, -1):
        i = random.randrange(remaining)
        yield state.get(i, i)
        state[i] = state.get(remaining - 1, remaining - 1)
        state.pop(remaining - 1, None)

def randomSet(S,n=0):
    Set = []
    if (n==0):
        n = len(S)
    rand = random.randint(1,n)
    return np.random.choice(S, rand ,replace=False)

def InconCheck(S,D,T):
    incon_value = 0
    data = D[S]
    data['target'] = T
    # create dataframe
    data_no = data.loc[data['target']==0]
    data_no = data_no.drop('target',axis=1)
    data_yes = data.loc[data['target']==1]
    data_yes = data_yes.drop('target',axis=1)
    no_df = pd.DataFrame(data_no.groupby(data_no.columns.tolist(),as_index=False).size().reset_index(name='target_no'))
    yes_df = pd.DataFrame(data_yes.groupby(data_yes.columns.tolist(),as_index=False).size().reset_index(name='target_yes'))

    result_df = pd.merge(no_df, yes_df, how='outer', on=S)
    result_df = result_df.fillna(0)
    result_df['inc'] = result_df['target_no']

    for index, row in result_df.iterrows():
        if (row['target_yes']>row['target_no']):
            result_df.iloc[index,result_df.columns=='inc'] = row['target_no']
        else:
            result_df.iloc[index,result_df.columns=='inc'] = row['target_yes']
    
    incon_value = result_df['inc'].sum()
    incon_value/len(D)

    return incon_value

def lvf(MAX_TRIES, D, TARGET_COLUMN, gamma):
    C_best = len(D)
    S_best = D.columns.values
    M_sets=[]
    
    for i in range(1,MAX_TRIES):
        S = sorted(randomSet(D.columns.values,C_best))
        C = len(S)
        if (C<C_best):
            if(InconCheck(S,D,TARGET_COLUMN)<gamma):
                M_sets=[]
                S_best = S
                C_best = C
                M_sets.append(S_best)
                print("")
                print("current best", S_best)
        elif ((C==C_best) and (InconCheck(S,D,TARGET_COLUMN)<gamma)) :
            M_sets.append(S)
            print("also current best", S)
    return M_sets

In [170]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB # put NOTE into paper - not sure if correct bayes
from sklearn.model_selection import StratifiedKFold # put NOTE into paper - better as common kfold sampling
from sklearn.base import clone
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from functools import reduce

CI_95_FACTOR = 1.96

class BaseClassifier:
    
    def __init__(self, clf, clf_name, modality):
        self.clf = clf
        self.clf_name = clf_name
        self.modality = modality
        
    def fit(self, df_features, df_targets, verbose=True):
        self.clf = clone(self.clf) # reset any previously trained model
        np.random.seed(32143421)
        if verbose:
            print(f"Starting training for classifier {self.clf_name} and modality {self.modality}")
        self.clf.fit(df_features, df_targets)
        
    def predict(self, df_features, verbose=True):
        if verbose:
            print(f"Starting prediction for classifier {self.clf_name} and modality {self.modality}")
        return self.clf.predict(df_features)    
        
class ClassifierFactory:
    
    @staticmethod
    def get_metadata_classifiers():
        return [BaseClassifier(KNeighborsClassifier(), 'k-Nearest neighbors', 'Metadata'),
                BaseClassifier(NearestCentroid(), 'Nearest mean classifier', 'Metadata'),
                BaseClassifier(DecisionTreeClassifier(), 'Decision tree', 'Metadata'),
                BaseClassifier(LogisticRegression(), 'Logistic regression', 'Metadata'),
                BaseClassifier(SVC(gamma='auto'), 'SVM (Gaussian Kernel)', 'Metadata'),
                BaseClassifier(BaggingClassifier(),'Bagging', 'Metadata'),
                BaseClassifier(RandomForestClassifier(n_estimators=10),'Random Forest', 'Metadata'),
                BaseClassifier(AdaBoostClassifier(), 'AdaBoost', 'Metadata'),
                BaseClassifier(GradientBoostingClassifier(), 'Gradient Boosting Tree', 'Metadata')]
    
    @staticmethod
    def get_text_classifiers():
        return [BaseClassifier(GaussianNB(), 'Naive Bayes', 'Textual'),
                BaseClassifier(KNeighborsClassifier(), 'k-Nearest neighbors', 'Textual'),
                BaseClassifier(SVC(gamma='auto'), 'SVM (Gaussian Kernel)', 'Textual')]
    
    @staticmethod
    def get_visual_classifiers():
        return [BaseClassifier(KNeighborsClassifier(), 'k-Nearest neighbors', 'Visual'),
                BaseClassifier(DecisionTreeClassifier(), 'Decision tree', 'Visual'),
                BaseClassifier(LogisticRegression(), 'Logistic regression', 'Visual'),
                BaseClassifier(SVC(gamma='auto'), 'SVM (Gaussian Kernel)', 'Visual'),
                BaseClassifier(RandomForestClassifier(n_estimators=10), 'Random Forest', 'Visual'),
                BaseClassifier(AdaBoostClassifier(), 'AdaBoost', 'Visual'),
                BaseClassifier(GradientBoostingClassifier(), 'Gradient Boosting Tree', 'Visual')]
    
    @staticmethod
    def get_audio_classifiers():
        return [BaseClassifier(LogisticRegression(), 'Logistic regression', 'Audio'),
                BaseClassifier(GradientBoostingClassifier(), 'Gradient Boosting Tree', 'Audio')]                
                
class DataWrapper:
    
    def __init__(self, df_audio, df_vis, df_txt, df_meta, df_targets):
        self.df_audio = df_audio
        self.df_vis = df_vis
        self.df_txt = df_txt
        self.df_meta = df_meta
        self.df_targets = df_targets
        
    def generate_subspace(self):
        # TODO insert LVW functionality
        # N = len(df_meta)
        # MAX_TRIES = 77*N^5
        # print(lvf(MAX_TRIES,df_meta, df_targets_train,5))
        
        self.df_audio = self.df_audio.drop('file_name', axis=1)
        self.df_vis = self.df_vis.drop('file_name', axis=1)
        self.df_txt = self.df_txt.drop('file_name', axis=1)
        self.df_meta = self.df_meta.drop('file_name', axis=1)
        
        
class OutputWrapper:
    
    def __init__(self, df_res_test, df_score_test, df_score_stats, df_res_all):
        self.df_res_test = df_res_test
        self.df_score_test = df_score_test
        self.df_score_stats = df_score_stats
        self.df_res_all = df_res_all
        
class Evaluator:
    # TODO add clf stacking method
    
    def __init__(self, data_wrapper, use_audio=True, use_visual=True, use_text=True, use_meta=True):
        self.data_wrapper = data_wrapper
        self.use_audio = use_audio
        self.use_visual = use_visual
        self.use_text = use_text
        self.use_meta = use_meta            
            
    def cv_modality(self, df_features, df_targets, clfs, cv=10, verbose=True, predict_all = False):
        kf = StratifiedKFold(n_splits=cv, random_state=9832432)
        df_cvs = []
        df_all_cvs = []
        i=1
        for train_index, test_index in kf.split(df_features, df_targets):
            if verbose:
                print(f"Performing CV fold {i}..")
            i += 1
            X_train, X_test = df_features.iloc[train_index,:], df_features.iloc[test_index,:]
            y_train, y_test = df_targets[train_index], df_targets[test_index]
            
            df_res = pd.DataFrame(y_test)
            df_res.columns = ['TARGET']
            
            # init df for prediction of all entries
            df_res_all = pd.DataFrame(df_targets)
            df_res_all.columns = ['TARGET']
            
            for clf in clfs:
                clf.fit(X_train, y_train, verbose)
                y_pred = clf.predict(X_test, verbose)
                df_res[clf.clf_name+"_"+clf.modality] = y_pred
                
                if predict_all:
                    y_pred_all = clf.predict(df_features, verbose)
                    df_res_all[clf.clf_name+"_"+clf.modality] = y_pred_all
                
            df_cvs.append(df_res)
            df_all_cvs.append(df_res_all)
        return df_cvs, df_all_cvs
            
    def cv(self, cv=10, verbose=True, predict_all=False):
        """
            set predict_all to True to also include predictions for all data
        """
        df_cvs = []
        df_all_cvs = []
        df_targets = self.data_wrapper.df_targets
        if (self.use_audio):
            df_test, df_all = self.cv_modality(
                self.data_wrapper.df_audio,
                df_targets,
                ClassifierFactory.get_audio_classifiers(),
                cv,
                verbose,
                predict_all)
            df_cvs.append(df_test)
            df_all_cvs.append(df_all)
        if (self.use_visual):
            df_test, df_all = self.cv_modality(
                self.data_wrapper.df_vis,
                df_targets,
                ClassifierFactory.get_visual_classifiers(),
                cv,
                verbose,
                predict_all)
            df_cvs.append(df_test)
            df_all_cvs.append(df_all)
        if (self.use_text):
            df_test, df_all = self.cv_modality(
                self.data_wrapper.df_txt,
                df_targets,
                ClassifierFactory.get_text_classifiers(),
                cv,
                verbose,
                predict_all)
            df_cvs.append(df_test)
            df_all_cvs.append(df_all)
        if (self.use_meta):
            df_test, df_all = self.cv_modality(
                self.data_wrapper.df_meta,
                df_targets,
                ClassifierFactory.get_metadata_classifiers(),
                cv,
                verbose,
                predict_all)
            df_cvs.append(df_test)
            df_all_cvs.append(df_all)
            
        df_c = []
        for i in range(len(df_cvs[0])):
            df_i = pd.concat([df_cvs[x][i] for x in range(len(df_cvs))], axis=1)
            df_i = df_i.loc[:,~df_i.columns.duplicated()]
            df_c.append(df_i)
            
        df_all_c = []
        if predict_all:
            for i in range(len(df_all_cvs[0])):
                df_i = pd.concat([df_all_cvs[x][i] for x in range(len(df_all_cvs))], axis=1)
                df_i = df_i.loc[:,~df_i.columns.duplicated()]
                df_all_c.append(df_i)
            
        df_res = self.evaluate(df_c)
        
        return OutputWrapper(df_c, df_res, self.overall_eval(df_res), df_all_c)

    def evaluate(self, df_res):
        """
        returns precision, recall and F1 in a DF
        returns list of DFs if df_res is list of DFs
        """
        if type(df_res) == type([]):
            return [self.evaluate_single(df_x) for df_x in df_res]
        else:
            return self.evaluate_single(df_res)

    def evaluate_single(self, df_res):
        cols = list(df_res.columns)
        cols.remove('TARGET')
        df_ev = pd.DataFrame(columns=['clf', 'precision', 'recall', 'f1'])
        for col in cols:
            prec = precision_score(df_res['TARGET'], df_res[col])
            recall = recall_score(df_res['TARGET'], df_res[col])
            f1 = f1_score(df_res['TARGET'], df_res[col])
            df_ev = df_ev.append({'clf':col, 'precision': prec, 'recall':recall, 'f1':f1}, ignore_index=True)
        return df_ev
    
    def overall_eval(self, df_results):
        df_score = pd.DataFrame(columns =['clf', 'mean_precision', 'mean_recall', 'mean_f1', 'var_precision', 'var_recall', 'var_f1', 'std_precision', 'std_recall', 'std_f1'])
        df_score['clf'] = df_results[0]['clf']
        # mean 
        df_res = reduce(lambda x, y: x.add(y, fill_value=0), df_results)
        df_score[['mean_precision', 'mean_recall', 'mean_f1']] = df_res[['precision', 'recall', 'f1']].div(len(df_results))

        # var
        df_mean = df_score[['mean_precision', 'mean_recall', 'mean_f1']]
        df_mean.columns = ['precision', 'recall', 'f1']
        df_dev = []
        for df in df_results:
            df_dev.append(np.square(df[['precision', 'recall', 'f1']].subtract(df_mean)))
        df_dev = reduce(lambda x, y: x.add(y, fill_value=0), df_dev)
        df_score[['var_precision', 'var_recall', 'var_f1']] = df_dev[['precision', 'recall', 'f1']].div(len(df_results))

        # std 
        df_score[['std_precision', 'std_recall', 'std_f1']] = np.power(df_score[['var_precision', 'var_recall', 'var_f1']], 0.5)
        
        # 95-ci
        for metric in ['precision', 'recall', 'f1']:
            l_ci = df_score['mean_'+metric] - df_score['std_'+metric]*CI_95_FACTOR/len(df_results)
            u_ci = df_score['mean_'+metric] + df_score['std_'+metric]*CI_95_FACTOR/len(df_results)
            df_score['l_95ci_'+metric] = l_ci
            df_score['u_95ci_'+metric] = u_ci

        return df_score

## Call Example

In [171]:
dw = DataWrapper(df_audio_train, df_vis_train, df_txt_train, df_meta_train, df_targets_train)
dw.generate_subspace()
ev = Evaluator(dw) # TODO preprocessing for meta needs to be done
ow = ev.cv(verbose=False, predict_all=True)

D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda3\lib\site-packages

In [175]:
ow.df_res_test[0]

,TARGET,Logistic regression_Audio,Gradient Boosting Tree_Audio,k-Nearest neighbors_Visual,Decision tree_Visual,Logistic regression_Visual,SVM (Gaussian Kernel)_Visual,Random Forest_Visual,AdaBoost_Visual,Gradient Boosting Tree_Visual,...,SVM (Gaussian Kernel)_Textual,k-Nearest neighbors_Metadata,Nearest mean classifier_Metadata,Decision tree_Metadata,Logistic regression_Metadata,SVM (Gaussian Kernel)_Metadata,Bagging_Metadata,Random Forest_Metadata,AdaBoost_Metadata,Gradient Boosting Tree_Metadata
0,1,1,0,1,0,0,1,0,0,0,...,1,1,1,1,1,1,0,1,1,1
1,1,1,0,1,1,1,1,0,1,1,...,1,0,0,0,0,0,0,0,1,0
2,1,1,1,1,0,1,1,0,0,0,...,1,1,1,1,1,1,1,0,1,1
3,1,1,0,1,0,0,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
4,0,1,0,1,1,1,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
5,1,1,1,1,0,0,1,0,1,1,...,1,0,0,0,0,0,0,1,0,0
6,1,1,0,0,0,0,1,0,1,0,...,1,0,1,0,0,1,0,0,0,0
7,1,1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,1,1,1
8,1,0,0,1,0,0,1,0,0,0,...,1,1,0,1,0,1,0,0,0,0
9,0,0,1,0,0,1,1,0,0,0,...,1,0,1,1,1,1,1,1,0,1


In [12]:
ow.df_score_test[0]        

,clf,precision,recall,f1
0,Logistic regression_Audio,0.555556,0.833333,0.666667
1,Gradient Boosting Tree_Audio,0.500000,0.333333,0.400000
2,k-Nearest neighbors_Visual,0.444444,0.666667,0.533333
3,Decision tree_Visual,0.600000,0.500000,0.545455
4,Logistic regression_Visual,0.571429,0.666667,0.615385
5,SVM (Gaussian Kernel)_Visual,0.545455,1.000000,0.705882
6,Random Forest_Visual,0.714286,0.833333,0.769231
7,AdaBoost_Visual,0.625000,0.833333,0.714286
8,Gradient Boosting Tree_Visual,0.666667,1.000000,0.800000
9,Naive Bayes_Textual,0.333333,0.333333,0.333333


In [13]:
ow.df_score_stats

,clf,mean_precision,mean_recall,mean_f1,var_precision,var_recall,var_f1,std_precision,std_recall,std_f1,l_95ci_precision,u_95ci_precision,l_95ci_recall,u_95ci_recall,l_95ci_f1,u_95ci_f1
0,Logistic regression_Audio,0.566508,0.570000,0.547244,0.047184,0.048989,0.030090,0.217218,0.221334,0.173464,0.523933,0.609083,0.526618,0.613382,0.513245,0.581243
1,Gradient Boosting Tree_Audio,0.486429,0.416667,0.438236,0.050547,0.054944,0.045175,0.224826,0.234402,0.212543,0.442363,0.530494,0.370724,0.462610,0.396578,0.479895
2,k-Nearest neighbors_Visual,0.568492,0.633333,0.588811,0.015035,0.035778,0.017594,0.122618,0.189150,0.132641,0.544459,0.592525,0.596260,0.670407,0.562814,0.614809
3,Decision tree_Visual,0.550476,0.623333,0.581818,0.023185,0.043567,0.030200,0.152265,0.208726,0.173782,0.520632,0.580320,0.582423,0.664244,0.547757,0.615879
4,Logistic regression_Visual,0.630952,0.656667,0.627471,0.021983,0.026233,0.012144,0.148267,0.161967,0.110201,0.601892,0.660013,0.624921,0.688412,0.605872,0.649070
5,SVM (Gaussian Kernel)_Visual,0.543434,0.983333,0.699755,0.000481,0.002500,0.000820,0.021922,0.050000,0.028633,0.539138,0.547731,0.973533,0.993133,0.694143,0.705367
6,Random Forest_Visual,0.619762,0.653333,0.629573,0.006054,0.019600,0.008414,0.077811,0.140000,0.091730,0.604511,0.635013,0.625893,0.680773,0.611594,0.647552
7,AdaBoost_Visual,0.584643,0.610000,0.593169,0.018714,0.025789,0.019354,0.136799,0.160589,0.139118,0.557830,0.611455,0.578525,0.641475,0.565902,0.620436
8,Gradient Boosting Tree_Visual,0.594524,0.726667,0.646410,0.009215,0.032400,0.012671,0.095997,0.180000,0.112566,0.575708,0.613339,0.691387,0.761947,0.624347,0.668473
9,Naive Bayes_Textual,0.535000,0.620000,0.550276,0.036233,0.055156,0.022024,0.190351,0.234852,0.148407,0.497691,0.572309,0.573969,0.666031,0.521189,0.579364


In [174]:
ow.df_res_all[1]

,TARGET,Logistic regression_Audio,Gradient Boosting Tree_Audio,k-Nearest neighbors_Visual,Decision tree_Visual,Logistic regression_Visual,SVM (Gaussian Kernel)_Visual,Random Forest_Visual,AdaBoost_Visual,Gradient Boosting Tree_Visual,...,SVM (Gaussian Kernel)_Textual,k-Nearest neighbors_Metadata,Nearest mean classifier_Metadata,Decision tree_Metadata,Logistic regression_Metadata,SVM (Gaussian Kernel)_Metadata,Bagging_Metadata,Random Forest_Metadata,AdaBoost_Metadata,Gradient Boosting Tree_Metadata
0,1,1,1,1,0,0,1,1,0,0,...,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,0,1,1,0,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,0,1,0,1,0,0,0,0,0,0,...,1,1,1,0,1,1,0,0,1,1
5,1,1,1,1,1,0,1,1,1,1,...,1,1,0,1,1,0,1,1,1,1
6,1,1,1,0,1,1,1,1,1,1,...,1,0,1,1,1,0,1,1,0,1
7,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
8,1,0,1,1,1,1,1,1,1,1,...,1,1,0,1,1,1,0,1,1,1
9,0,0,0,1,0,1,0,0,0,0,...,1,0,1,0,0,1,0,0,0,0


In [177]:
def majorityVoting2(cvs_results):
    results=[]
    for fold in range(0,len(cvs_results[0])):
        df_vars=cvs_results[0][fold]
        df_labels=cvs_results[0][fold]['TARGET']
        df_labels=df_labels.reset_index(drop=True)
        for x in range(1,len(cvs_results)):
            tmp=cvs_results[x][fold]
            tmp=tmp.drop(['TARGET'],axis = 1)
            df_vars=pd.concat([df_vars,tmp],axis = 1)
            
        res=df_vars.mode(axis=1)[0]
        res=res.reset_index(drop=True)
        res=pd.to_numeric(res)
        df_labels=pd.to_numeric(df_labels)   
        
        #results.append(res)
        precision = precision_score(df_labels, res)
        recall = recall_score(df_labels,res)
        f1 = f1_score(df_labels,res)
        results.append([precision,recall,f1])
        
        #print(precision,recall,f1)
        
    return results
   
def majorityVoting(cvs_results,droppedClfs):
    results=[]
    for fold in range(0,len(cvs_results)):
        df_vars=cvs_results[fold]
        df_vars=df_vars.drop(['TARGET'],axis=1)
        
        for clf in droppedClfs:
            df_vars= df_vars.drop([clf],axis = 1)
        
        df_labels=cvs_results[fold]['TARGET']
        
        res=df_vars.mode(axis=1)[0]
        res=pd.to_numeric(res)
        df_labels=pd.to_numeric(df_labels)   
        
        precision = precision_score(df_labels, res)
        recall = recall_score(df_labels,res)
        f1 = f1_score(df_labels,res)
        results.append([precision,recall,f1])
        
    return results
   
    
    
def labelStacking(df_all_folds,df_test_folds,droppedClfs):
    results=[]
    for fold in range(0,len(df_all_folds)):
        df_all = df_all_folds[fold]
        df_test = df_test_folds[fold]
        df_new=df_all.merge(df_test, how='left', indicator=True)
        df_train = df_new[df_new['_merge'] == 'left_only']
        #print(df_train)
        df_train = df_train.drop(['_merge'],axis = 1)
        #clf=SVC(gamma='auto')
        #clf=LogisticRegression()
        #clf=DecisionTreeClassifier()
        #clf=NearestCentroid()
        #clf=RandomForestClassifier(n_estimators=100)
        clf=KNeighborsClassifier()
        #clf=AdaBoostClassifier()
        #clf=GradientBoostingClassifier()
        X_train = df_train.loc[:, df_train.columns != 'TARGET']
        y_train = df_train['TARGET']
        
        X_test = df_test.loc[:, df_test.columns != 'TARGET']
        y_test = df_test['TARGET']
        
        for c in droppedClfs:
            X_train= X_train.drop([c],axis = 1)
            X_test= X_test.drop([c],axis = 1)
            
        model = clf.fit(X_train, y_train)
        precision = precision_score(y_test,model.predict(X_test))
        recall = recall_score(y_test,model.predict(X_test))
        f1 = f1_score(y_test,model.predict(X_test))
        results.append([precision,recall,f1])
    
    return results

def labelFeatureStacking(df_all_folds,df_test_folds,droppedClfs):
    results=[]
    for fold in range(0,len(df_all_folds)):
        df_all = df_all_folds[fold]
        df_test = df_test_folds[fold]
        df_new=df_all.merge(df_test, how='left', indicator=True)
        df_train = df_new[df_new['_merge'] == 'left_only']
        df_train = df_train.drop(['_merge'],axis = 1)
        
        df_combined_train=pd.merge(df_train, df_vis_train, left_index=True, right_index=True)
        df_combined_train = df_combined_train.drop(['file_name'],axis = 1)
        df_combined_test=pd.merge(df_test, df_vis_train, left_index=True, right_index=True)
        df_combined_test = df_combined_test.drop(['file_name'],axis = 1)
        
        X_train = df_combined_train.loc[:, df_combined_train.columns != 'TARGET']
        y_train = df_combined_train['TARGET']
        
        X_test = df_combined_test.loc[:, df_combined_test.columns != 'TARGET']
        y_test = df_combined_test['TARGET']
        for c in droppedClfs:
            X_train= X_train.drop([c],axis = 1)
            X_test= X_test.drop([c],axis = 1)
        
                                  
        clf=SVC(gamma='auto')
        #clf=LogisticRegression()
        #clf=DecisionTreeClassifier()
        #clf=NearestCentroid()
        #clf=RandomForestClassifier(n_estimators=100)
        #clf=KNeighborsClassifier()
        #clf=AdaBoostClassifier()
        #clf=GradientBoostingClassifier()
                                  
        model = clf.fit(X_train, y_train)
        precision = precision_score(y_test,model.predict(X_test))
        recall = recall_score(y_test,model.predict(X_test))
        f1 = f1_score(y_test,model.predict(X_test))
        results.append([precision,recall,f1])
    

    return results
    
def calcResults(resMat):
    npres = np.asarray(resMat)
    meanVals=np.mean(npres,axis=0)
    varVals=np.var(npres,axis=0)
    stdVals=np.std(npres,axis=0)
    CI_95_FACTOR = 1.96
    l_ci = meanVals - stdVals*CI_95_FACTOR/npres.shape[0]
    u_ci = meanVals + stdVals*CI_95_FACTOR/npres.shape[0]
    
    print("Mean Values: ",meanVals)
    print("Variance Values: ",varVals)
    print("Std Values: ",stdVals)
    print("95%-CI:\nPrecision {}-{}; Recall {}-{}; F1 {}-{};".format(l_ci[0],u_ci[0],l_ci[1],u_ci[1],l_ci[2],u_ci[2]))



filtered_p = ow.df_score_stats['clf'][ow.df_score_stats['mean_precision']<0.5]
filtered_r = ow.df_score_stats['clf'][ow.df_score_stats['mean_recall']<0.5]
filtered_f = ow.df_score_stats['clf'][ow.df_score_stats['mean_f1']<0.5]
df_filtered =pd.concat([filtered_p,filtered_r],axis = 0)
df_filtered =pd.concat([df_filtered,filtered_f],axis=0)
droppedClfs=df_filtered.drop_duplicates()

res1=majorityVoting(ow.df_res_test,droppedClfs)
print("-"*50)
print("Majority Voting Results:")
averages1=calcResults(res1)

#print("Majority Voting:")
#print("CV Scores: ",res1)
#print("AVG: ",averages1)

res2=labelStacking(ow.df_res_all,ow.df_res_test,droppedClfs)
print("-"*50)
print("Label Stacking Results:")
averages2=calcResults(res2)


#print("Label Stacking:")
#print("CV Scores: ",res2)
#print("AVG: ",averages2)

res3=labelFeatureStacking(ow.df_res_all,ow.df_res_test,droppedClfs)
print("-"*50)
print("Label Attribute Stacking Results:")
averages3=calcResults(res3)
                                  

#print("Label-Feature Stacking:")
#print("CV Scores: ",res3)
#print("AVG: ",averages3)


--------------------------------------------------
Majority Voting Results:
Mean Values:  [0.60745713 0.72272727 0.65133344]
Variance Values:  [0.00550447 0.01688705 0.00523166]
Std Values:  [0.0741921  0.12995019 0.07233022]
95%-CI:
Precision 0.5929154736721126-0.6219987787421398; Recall 0.6972570351399471-0.7481975103145985; F1 0.6371567184619235-0.6655101661408876;
--------------------------------------------------
Label Stacking Results:
Mean Values:  [0.60834356 0.65227273 0.61756822]
Variance Values:  [0.01057294 0.03077881 0.0138641 ]
Std Values:  [0.10282479 0.17543891 0.11774592]
95%-CI:
Precision 0.5881898984436841-0.628497215611851; Recall 0.6178867010838592-0.6866587534615954; F1 0.5944900223336272-0.6406464242208542;
--------------------------------------------------
Label Attribute Stacking Results:
Mean Values:  [0.53844687 0.89621212 0.67123546]
Variance Values:  [0.00255192 0.01230544 0.00419044]
Std Values:  [0.05051655 0.11092989 0.0647336 ]
95%-CI:
Precision 0.52854